First, create the model. This must match the model used in the interactive training notebook.

In [1]:
def mask_img(img, cut_ratio=0.35):
    img[:int(img.shape[0]*cut_ratio), :, :] = 0

In [2]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [3]:
model.load_state_dict(torch.load('road_following_model.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [4]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [ ]:
torch.save(model_trt.state_dict(), 'road_following_model_trt.pth')

Load the optimized model by executing the cell below

In [6]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

Create the racecar class

In [14]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [17]:
# from jetcam.csi_camera import CSICamera
from jetcam.usb_camera import USBCamera
# camera = CSICamera(width=224, height=224, capture_fps=65)
camera = USBCamera(width=224, height=224)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [18]:
from utils import preprocess
import numpy as np

# STEERING_GAIN = 0.75
# STEERING_BIAS = 0.00

STEERING_GAIN = -1.0
STEERING_BIAS = 0.00

car.steering_gain = 1
car.steering_offset = 0
car.throttle_gain = 1
car.throttle = 0.3

image = camera.read()
mask_img(image)
image = preprocess(image).half()
output = model_trt(image).detach().cpu().numpy().flatten()
x = float(output[0])
car.steering = x * STEERING_GAIN + STEERING_BIAS
print(x)

Twisted reactor is already running


-0.111328125
